In [1]:

import json
import os

res_folder = "C:\\Users\\PC\\CODE\\WDM-AI-TEMIS\\data-finetune\\final_data"
json_res = os.path.join(res_folder, "final.json")
docling_json = os.path.join(res_folder, "docling_json.json")
image_folder = "C:\\Users\\PC\\CODE\\WDM-AI-TEMIS\\data-finetune\\final_data\\extracted_images"
pdf_paths = "C:\\Users\\PC\\CODE\\WDM-AI-TEMIS\\data-finetune\\pdf4tabel"
res = json.load(open(json_res, 'r', encoding='utf-8'))
pymupdf_json = os.path.join(res_folder, "pymupdf_json.json")

new_res = []

In [4]:
from PIL import Image

def take_image(image_path):
    image_path = os.path.join(image_folder, image_path)
    image = Image.open(image_path)
    return image

In [116]:
image = take_image(res[0]['image_path'])
image.show()

## Docling

In [54]:
import time

from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode
from docling.document_converter import DocumentConverter, PdfFormatOption

def docling_md(source):
    # source = os.path.join(image_folder, res[0]['image_path'])
    converter = DocumentConverter()
    result = converter.convert(source)
    return result.document.export_to_markdown()

In [ ]:
for idx, sample in enumerate(res):
    image_path = sample['image_path']
    image_path = os.path.join(image_folder, image_path)
    if os.path.exists(image_path):
        sample['docling_md'] = docling_md(image_path)
        res[idx]['docling_md'] = sample['docling_md']
    else:
        sample['docling_md'] = "Cannot extract image"
    break

## PymuPDF

In [2]:
# Cần cài đặt các thư viện nếu chưa có:
# pip install python-Levenshtein scikit-learn numpy

import Levenshtein
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --- Các hàm tính độ tương đồng (Giữ nguyên) ---

import re

def normalize_markdown_for_comparison(md_string):
    if not isinstance(md_string, str):
        return ""
    # 1. Thay thế <br> hoặc <br /> bằng một khoảng trắng (hoặc newline nếu bạn muốn giữ cấu trúc nhiều dòng trong ô)
    #    Ở đây, tôi thay bằng khoảng trắng để các ô thành một dòng dài hơn.
    processed_string = re.sub(r'<br\s*/?>', ' ', md_string)

    # 2. (Tùy chọn) Xử lý các ký tự đặc biệt. Ví dụ, nếu PyMuPDF loại bỏ ☑:
    #    Nếu bạn muốn so sánh có phân biệt ☑, bạn cần tìm cách giữ nó trong PyMuPDF output (khó)
    #    Hoặc, loại bỏ nó khỏi cả hai chuỗi để so sánh công bằng hơn (mất thông tin)
    #    processed_string = processed_string.replace("☑", "") # Ví dụ loại bỏ

    # 3. Chuẩn hóa nhiều khoảng trắng thành một khoảng trắng
    processed_string = re.sub(r'\s+', ' ', processed_string)

    # 4. Strip từng dòng và kết hợp lại (quan trọng sau các thay đổi)
    lines = [line.strip() for line in processed_string.split('\n') if line.strip()] # Lọc dòng trống
    return "\n".join(lines)

def levenshtein_similarity(str1, str2):
    distance = Levenshtein.distance(str1, str2)
    max_len = max(len(str1), len(str2))
    if max_len == 0:
        return 1.0
    return 1.0 - (distance / max_len)

def jaccard_similarity_lines(table1_str, table2_str):
    """
    Tính độ tương đồng Jaccard dựa trên các dòng của hai bảng.
    Đã cải thiện để robust hơn với khoảng trắng ở đầu/cuối dòng và các dòng trống.
    """
    lines1 = {line.strip() for line in table1_str.strip().split('\n') if line.strip()}
    lines2 = {line.strip() for line in table2_str.strip().split('\n') if line.strip()}

    if not lines1 and not lines2:
        return 1.0
    if not lines1 or not lines2:
        return 0.0

    intersection = len(lines1.intersection(lines2))
    union = len(lines1.union(lines2))

    if union == 0: # Về lý thuyết không đạt tới đây nếu đã qua các check trên
        return 1.0 
    return intersection / union

def calculate_cosine_similarity_for_list(table_list_to_compare, ground_truth_table):
    if not table_list_to_compare:
        return np.array([])

    documents = [ground_truth_table] + table_list_to_compare

    try:
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(documents)
    except ValueError:
        similarities = []
        for doc_str_in_list in table_list_to_compare:
            gt_is_empty = not ground_truth_table.strip()
            doc_is_empty = not doc_str_in_list.strip()
            if gt_is_empty and doc_is_empty:
                 similarities.append(1.0)
            elif (gt_is_empty and not doc_is_empty) or (not gt_is_empty and doc_is_empty):
                 similarities.append(0.0)
            else:
                 similarities.append(0.0)
        return np.array(similarities)

    if tfidf_matrix.shape[0] < 2 :
        return np.zeros(len(table_list_to_compare))

    ground_truth_vector = tfidf_matrix[0]
    other_vectors = tfidf_matrix[1:]

    if other_vectors.shape[0] == 0:
        return np.zeros(len(table_list_to_compare))

    similarities = cosine_similarity(ground_truth_vector, other_vectors)
    return similarities.flatten()

# --- Hàm Ensemble Chính (Cập nhật với Ngưỡng) ---

def find_most_similar_table_ensemble(label_table_str, list_of_table_strs, verbose=True, min_similarity_threshold=None):
    """
    Tìm chỉ số của bảng trong list_of_table_strs giống nhất với label_table_str
    sử dụng phương pháp ensemble (trung bình) các điểm tương đồng.
    Nếu không có bảng nào đạt ngưỡng tương đồng tối thiểu, trả về -1.

    Args:
        label_table_str (str): Chuỗi Markdown của bảng ground truth.
        list_of_table_strs (list): Danh sách các chuỗi Markdown của các bảng cần so sánh.
        verbose (bool): Nếu True, in ra các điểm tương đồng chi tiết.
        min_similarity_threshold (float, optional): Ngưỡng tương đồng trung bình tối thiểu
                                                     để một bảng được coi là khớp.
                                                     Nếu None, không áp dụng ngưỡng (trả về chỉ số tốt nhất).
                                                     Mặc định là None.

    Returns:
        int: Chỉ số của bảng giống nhất nếu đạt ngưỡng.
             Trả về -1 nếu không có bảng nào đạt ngưỡng, danh sách rỗng, hoặc có lỗi.
    """
    if not all(isinstance(s, str) for s in [label_table_str] + list_of_table_strs):
        if verbose:
            print("Lỗi: Không phải tất cả đầu vào đều là chuỗi. Hãy đảm bảo label và tất cả các bảng trong danh sách là chuỗi.")
        return -1

    if not list_of_table_strs:
        if verbose:
            print("Danh sách bảng để so sánh trống.")
        return -1

    num_tables = len(list_of_table_strs)
    levenshtein_scores = np.zeros(num_tables)
    jaccard_scores = np.zeros(num_tables)

    if verbose:
        print("--- Levenshtein Similarity ---")
    for i, table_content in enumerate(list_of_table_strs):
        sim = levenshtein_similarity(label_table_str, table_content)
        levenshtein_scores[i] = sim
        if verbose:
            print(f"Bảng {i} vs ground truth: {sim:.4f}")
    if num_tables > 0:
        best_levenshtein_idx = np.argmax(levenshtein_scores)
        if verbose:
            print(f"Chỉ số bảng giống nhất (Levenshtein): {best_levenshtein_idx} với điểm {levenshtein_scores[best_levenshtein_idx]:.4f}\n")

    if verbose:
        print("--- Jaccard Similarity (dựa trên các dòng) ---")
    for i, table_content in enumerate(list_of_table_strs):
        sim = jaccard_similarity_lines(label_table_str, table_content)
        jaccard_scores[i] = sim
        if verbose:
            print(f"Bảng {i} vs ground truth: {sim:.4f}")
    if num_tables > 0:
        best_jaccard_idx = np.argmax(jaccard_scores)
        score_to_display_jaccard = jaccard_scores[best_jaccard_idx] if jaccard_scores.size > 0 else 0.0
        if verbose:
            print(f"Chỉ số bảng giống nhất (Jaccard - dòng): {best_jaccard_idx} với điểm {score_to_display_jaccard:.4f}\n")

    if verbose:
        print("--- Cosine Similarity (TF-IDF) ---")
    cosine_scores = calculate_cosine_similarity_for_list(list_of_table_strs, label_table_str)
    if cosine_scores.size == num_tables:
        for i, score in enumerate(cosine_scores):
            if verbose:
                print(f"Bảng {i} vs ground truth: {score:.4f}")
        best_cosine_idx = np.argmax(cosine_scores) if cosine_scores.size > 0 else 0
        score_to_display_cosine = cosine_scores[best_cosine_idx] if cosine_scores.size > 0 else 0.0
        if verbose:
             print(f"Chỉ số bảng giống nhất (Cosine TF-IDF): {best_cosine_idx} với điểm {score_to_display_cosine:.4f}\n")
    else:
        if verbose:
            print("Không thể tính toán Cosine scores một cách chính xác, sử dụng điểm 0 cho ensemble.\n")
        cosine_scores = np.zeros(num_tables)

    if not (len(levenshtein_scores) == num_tables and \
            len(jaccard_scores) == num_tables and \
            len(cosine_scores) == num_tables):
        if verbose:
            print("Lỗi: Kích thước của các mảng điểm không khớp để tính trung bình.")
        return -1

    average_scores = (levenshtein_scores + jaccard_scores + cosine_scores) / 3.0
    
    if verbose:
        print("--- Điểm Trung Bình ---")
        for i, avg_score in enumerate(average_scores):
            print(f"Bảng {i} - Điểm trung bình: {avg_score:.4f}")

    if average_scores.size > 0:
        best_average_idx = np.argmax(average_scores)
        highest_avg_score = average_scores[best_average_idx]

        if verbose:
            print(f"\nChỉ số bảng có điểm trung bình cao nhất: {best_average_idx} với điểm trung bình {highest_avg_score:.4f}")
            if min_similarity_threshold is not None:
                 print(f"Ngưỡng tương đồng tối thiểu yêu cầu: {min_similarity_threshold:.4f}")

        if min_similarity_threshold is not None:
            if highest_avg_score >= min_similarity_threshold:
                if verbose:
                    print(f"Điểm trung bình cao nhất ({highest_avg_score:.4f}) >= ngưỡng ({min_similarity_threshold:.4f}). Trả về chỉ số.")
                return int(best_average_idx)
            else:
                if verbose:
                    print(f"Điểm trung bình cao nhất ({highest_avg_score:.4f}) < ngưỡng ({min_similarity_threshold:.4f}). Không có bảng nào đủ giống. Trả về -1.")
                return -1
        else: # Không có ngưỡng, trả về chỉ số tốt nhất
            if verbose:
                print("Không có ngưỡng tương đồng tối thiểu nào được áp dụng. Trả về chỉ số tốt nhất.")
            return int(best_average_idx)
    else:
        if verbose:
            print("\nKhông thể tính điểm trung bình tổng hợp hoặc không có bảng nào để đánh giá.")
        return -1


In [13]:
import pymupdf


def get_pdf_name(source: str) -> str:
    pdf_name = os.path.basename(source)
    file_name_part, file_extension_part = os.path.splitext(pdf_name)
    return file_name_part

def pymupdf_md(sample): 
    pdf_name = get_pdf_name(sample['source_pdf'])
    source_pdf_path = os.path.join(pdf_paths, pdf_name+".pdf")

    doc = pymupdf.open(source_pdf_path)
    extracted_raw_markdown_tables = []
    for page_num, page in enumerate(doc): # Thêm page_num để debug nếu cần
        try:
            # Cân nhắc các strategy khác nhau: "lines", "lines_strict", "text"
            # find_tables() trả về một TableFinder object
            table_finder = page.find_tables(strategy="lines_strict") 
            if table_finder.tables: # Kiểm tra xem có bảng nào được tìm thấy không
                for table in table_finder.tables:
                    extracted_raw_markdown_tables.append(table.to_markdown(clean=False)) # Thử nghiệm clean=False/True
        except Exception as e:
            print(f"Lỗi khi trích xuất bảng từ trang {page_num} của file {sample['source_pdf']}: {e}")
    doc.close()

    label_ground_truth = sample['markdown_content']
    normalized_label = normalize_markdown_for_comparison(label_ground_truth)

    # Chuẩn hóa các bảng đã trích xuất
    normalized_extracted_tables = [normalize_markdown_for_comparison(t) for t in extracted_raw_markdown_tables]

    # Lọc bỏ các chuỗi rỗng sau khi chuẩn hóa (nếu có)
    # và giữ index gốc để tham chiếu lại bảng gốc nếu cần
    valid_normalized_tables_with_original_indices = []
    for i, norm_table in enumerate(normalized_extracted_tables):
        if norm_table: # Chỉ xem xét các bảng không rỗng sau chuẩn hóa
            valid_normalized_tables_with_original_indices.append((norm_table, i))

    final_normalized_tables_to_compare = [item[0] for item in valid_normalized_tables_with_original_indices]


    if not final_normalized_tables_to_compare:
        # print(f"Không có bảng nào hợp lệ được trích xuất từ {sample['source_pdf']} sau chuẩn hóa.")
        sample['pymupdf_md'] = "No valid tables extracted or normalized"
        return sample

    # Gọi hàm ensemble với dữ liệu đã chuẩn hóa
    best_match_normalized_idx = find_most_similar_table_ensemble(
        normalized_label, 
        final_normalized_tables_to_compare, # Danh sách các bảng đã chuẩn hóa
        verbose=False, # Đặt True để debug nếu cần
        min_similarity_threshold=0.3 
    )

    if best_match_normalized_idx != -1:
        # Lấy index của bảng gốc từ PyMuPDF output
        original_table_index = valid_normalized_tables_with_original_indices[best_match_normalized_idx][1]
        table_selected_original_format = extracted_raw_markdown_tables[original_table_index]
        sample['pymupdf_md'] = table_selected_original_format
    else:
        sample['pymupdf_md'] = "No suitable table found meeting threshold"

    return sample 

In [ ]:
from tqdm import tqdm

# Process samples sequentially with progress bar
results = []
for sample in tqdm(res, total=len(res)):
    try:
        result = pymupdf_md(sample)
        results.append(result)
    except Exception as e:
        print(f"Error processing sample: {e}")
        results.append(None)

  0%|          | 0/163 [00:00<?, ?it/s]

MuPDF error: format error: cmsOpenProfileFromMem failed

MuPDF error: format error: cmsOpenProfileFromMem failed



  1%|          | 1/163 [00:13<35:42, 13.23s/it]

MuPDF error: format error: cmsOpenProfileFromMem failed

MuPDF error: format error: cmsOpenProfileFromMem failed



  1%|          | 2/163 [00:26<34:56, 13.02s/it]

...

 64%|██████▍   | 104/163 [25:09<08:12,  8.35s/it]

MuPDF error: format error: cmsOpenProfileFromMem failed



 64%|██████▍   | 105/163 [25:26<10:48, 11.18s/it]

MuPDF error: format error: cmsOpenProfileFromMem failed



 77%|███████▋  | 125/163 [33:51<16:41, 26.34s/it]

In [ ]:
#save json
pymupdf_json = os.path.join(res_folder, "pymupdf_json.json")
with open(pymupdf_json, 'w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4)